# Data Cleaning



In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.rendered_html { font-size: 18px; }</style>"))

First We will import the modules we need as well as the SQL shortened data sets

In [23]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import nltk
import datetime
import math
#from sqlalchemy import create_engine
#nltk.download()

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
raw_2016 = pd.read_csv('2016.csv')
raw_2017 = pd.read_csv('2017.csv')

In [25]:
combined_df = pd.concat([raw_2016,raw_2017]).copy()
#combined_df.set_index('parcelid', inplace=True)
print(len(combined_df['parcelid'].unique()),len(combined_df['parcelid']))
print(len(raw_2016['parcelid']),len(raw_2017['parcelid']),len(combined_df['parcelid']))
combined_df.head()

165210 167888
90275 77613 167888


,Unnamed: 0,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,...,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27
1,1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,0.0198,2016-03-30
2,2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.0060,2016-05-27
3,3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,...,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.0566,2016-06-07
4,4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.0573,2016-08-08


The SQL commands that created thses datasets plus the merging created a few vestigial columns which we will drop. We should have 58 data fields plus logerror and transaction date for a total of 60 columns.

In [26]:
combined_df = combined_df.drop('Unnamed: 0', 1)
#combined_df = combined_df.drop('index', 1)
#combined_df = combined_df.drop('index.1', 1)
#combined_df = combined_df.drop('parcelid.1', 1)

In [27]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167888 entries, 0 to 77612
Data columns (total 60 columns):
parcelid                        167888 non-null int64
airconditioningtypeid           53788 non-null float64
architecturalstyletypeid        468 non-null float64
basementsqft                    93 non-null float64
bathroomcnt                     167854 non-null float64
bedroomcnt                      167854 non-null float64
buildingclasstypeid             31 non-null float64
buildingqualitytypeid           107173 non-null float64
calculatedbathnbr               166056 non-null float64
decktypeid                      1272 non-null float64
finishedfloor1squarefeet        12893 non-null float64
calculatedfinishedsquarefeet    166992 non-null float64
finishedsquarefeet12            159519 non-null float64
finishedsquarefeet13            75 non-null float64
finishedsquarefeet15            6591 non-null float64
finishedsquarefeet50            12893 non-null float64
finishedsquarefeet

In [28]:
combined_df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,1.678880e+05,53788.000000,468.000000,93.000000,167854.000000,167854.000000,31.000000,107173.000000,166056.000000,1272.0,...,38169.000000,394.0,1.673590e+05,1.678520e+05,167854.000000,1.678510e+05,167843.000000,4683.000000,1.670020e+05,167888.000000
mean,1.299536e+07,1.814345,7.299145,695.376344,2.288265,3.041739,3.967742,6.015461,2.312542,66.0,...,1.437764,1.0,1.843460e+05,4.726833e+05,2015.462181,2.888805e+05,5989.515575,13.827247,6.049390e+13,0.013906
std,3.016071e+06,2.970239,2.719633,583.952144,1.000835,1.149134,0.179605,1.882799,0.977808,0.0,...,0.544508,0.0,2.192841e+05,6.028372e+05,0.498569,4.456451e+05,7214.731946,2.421658,1.054137e+12,0.165706
min,1.071174e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,66.0,...,1.000000,1.0,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,3.000000,6.037101e+13,-4.655420
25%,1.154899e+07,1.000000,7.000000,280.000000,2.000000,2.000000,4.000000,4.000000,2.000000,66.0,...,1.000000,1.0,8.256800e+04,2.026522e+05,2015.000000,8.373000e+04,2798.070000,13.000000,6.037312e+13,-0.025300
50%,1.254060e+07,1.000000,7.000000,588.000000,2.000000,3.000000,4.000000,7.000000,2.000000,66.0,...,1.000000,1.0,1.340510e+05,3.500000e+05,2015.000000,1.977760e+05,4501.040000,14.000000,6.037604e+13,0.006000
75%,1.421930e+07,1.000000,7.000000,819.000000,3.000000,4.000000,4.000000,7.000000,3.000000,66.0,...,2.000000,1.0,2.142570e+05,5.540020e+05,2016.000000,3.553340e+05,6914.165000,15.000000,6.059042e+13,0.039200
max,1.676893e+08,13.000000,21.000000,3560.000000,20.000000,16.000000,4.000000,12.000000,20.000000,66.0,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.000000,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


We will bring in some of our functions used to convert values, though use them after some null cleaning.

In [29]:
def str_to_intcat(df,col):
    result = pd.factorize(df[col])[0]
    return pd.Series((v for v in result))

def bool_to_int(df,col):
    result = []
    for entry in df[col]:
        if entry == 'Y' or entry == True:
            result.append(1)
        elif entry == 'N' or entry == False:
            result.append(0)
        else:
            result.append(-1)
    return pd.Series((v for v in result))
    

def to_datetime(df,col):
    result = []
    for entry in df[col]:
        if pd.isnull(entry) == True:
            result.append(entry)
        else:
            result.append(pd.Timestamp(entry))
    return pd.Series((v for v in result))

Now we will clean up some of the null values. First we will find a list of columns which have more than 50,000 non-null values, and use those as a means of keeping just those columns. Then we will find and remove those rows with fewer than 20 nun-null values.

This method is a messy method, and we will lose some columns in which missing values may not be missing but rather having null values indicating false or 0 values. We will address these seperately later.

In [30]:
columns_to_keep = []
for column in combined_df:
    if combined_df[column].count() > 50000:
        columns_to_keep.append(column)

#columns_to_keep

In [31]:
combined_df = combined_df.dropna(thresh=20)
len(combined_df)

167805

In [32]:
reduced_df = combined_df[columns_to_keep]
reduced_df.head()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate
0,17073783,NaN,2.5,3.0,NaN,2.5,1264.0,1264.0,6111.0,2.0,...,NaN,1986.0,115087.0,191811.0,2015.0,76724.0,2015.06,6.111002e+13,0.0953,2016-01-27
1,17088994,NaN,1.0,2.0,NaN,1.0,777.0,777.0,6111.0,1.0,...,NaN,1990.0,143809.0,239679.0,2015.0,95870.0,2581.30,6.111002e+13,0.0198,2016-03-30
2,17100444,NaN,2.0,3.0,NaN,2.0,1101.0,1101.0,6111.0,2.0,...,NaN,1956.0,33619.0,47853.0,2015.0,14234.0,591.64,6.111001e+13,0.0060,2016-05-27
3,17102429,NaN,1.5,2.0,NaN,1.5,1554.0,1554.0,6111.0,1.0,...,NaN,1965.0,45609.0,62914.0,2015.0,17305.0,682.78,6.111001e+13,-0.0566,2016-06-07
4,17109604,NaN,2.5,4.0,NaN,2.5,2415.0,2415.0,6111.0,2.0,...,NaN,1984.0,277000.0,554000.0,2015.0,277000.0,5886.92,6.111001e+13,0.0573,2016-08-08


In [33]:
reduced_df.describe()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror
count,1.678050e+05,53788.000000,167805.000000,167805.000000,107173.000000,166056.000000,166992.000000,159519.000000,167805.000000,166056.000000,...,167805.000000,109056.000000,166828.000000,1.673390e+05,1.678030e+05,167805.000000,1.678020e+05,167794.000000,1.670020e+05,167805.000000
mean,1.299132e+07,1.814345,2.288934,3.042627,6.015461,2.312542,1778.630246,1752.326939,6048.835690,2.245345,...,1.477912,1.110365,1968.569311,1.842978e+05,4.726717e+05,2015.462281,2.888853e+05,5989.373165,6.049390e+13,0.013914
std,2.920182e+06,2.970239,1.000217,1.148126,1.882799,0.977808,940.356025,921.360898,20.690315,0.964703,...,2.821794,0.988168,23.777202,2.187958e+05,6.025877e+05,0.498577,4.456502e+05,7212.175482,1.054137e+12,0.165487
min,1.071174e+07,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,6037.000000,1.000000,...,0.000000,1.000000,1824.000000,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,6.037101e+13,-4.655420
25%,1.154897e+07,1.000000,2.000000,2.000000,4.000000,2.000000,1183.000000,1172.000000,6037.000000,2.000000,...,0.000000,1.000000,1953.000000,8.257500e+04,2.027490e+05,2015.000000,8.374250e+04,2799.002500,6.037312e+13,-0.025300
50%,1.254063e+07,1.000000,2.000000,3.000000,7.000000,2.000000,1541.000000,1520.000000,6037.000000,2.000000,...,0.000000,1.000000,1970.000000,1.340550e+05,3.500000e+05,2015.000000,1.978240e+05,4501.435000,6.037604e+13,0.006000
75%,1.421900e+07,1.000000,3.000000,4.000000,7.000000,3.000000,2103.000000,2065.000000,6059.000000,3.000000,...,0.000000,1.000000,1987.000000,2.142500e+05,5.540090e+05,2016.000000,3.553370e+05,6914.630000,6.059042e+13,0.039200
max,1.676885e+08,13.000000,20.000000,16.000000,12.000000,20.000000,35640.000000,21929.000000,6111.000000,20.000000,...,18.000000,237.000000,2016.000000,1.142179e+07,4.906124e+07,2016.000000,4.895220e+07,586639.300000,4.830301e+14,5.262999


We still have a number of empty values but we will wait to use some statistical tools to evaluate their usefulness or fill them with estimated values.

In the meantime, we will need to convert a number of the float values to categorical int values, as well as a few object values to int or datetime.

It may not be necessary, but we will work on values that count something and pure categorical values separately even though we are mapping both to integers

In [34]:
int_cols = ['airconditioningtypeid', 'bathroomcnt', 'fullbathcnt', 'garagecarcnt', 'roomcnt', 'unitcnt']
cat_cols = ['airconditioningtypeid','buildingqualitytypeid','fips','heatingorsystemtypeid',
           'propertylandusetypeid','regionidcity','regionidcounty','regionidneighborhood','regionidzip']
obj_cols = ['propertycountylandusecode','propertyzoningdesc']
date_cols = ['yearbuilt','assessmentyear']
# check 'calculatedbathnbr' and 'garagecarcnt' for possible float values.
# check values for rawcensustractandblock and censustractandblock

#combined_df[cat_cols].head()

In [35]:
ques_cols = ['calculatedbathnbr', 'garagecarcnt']

for column in ques_cols:
    df = combined_df[column].unique()
    print(df)

[  2.5   1.    2.    1.5   3.    7.    5.    nan   4.    4.5   3.5   5.5
   6.   10.    8.    9.    6.5  12.   11.    7.5   8.5  20.   15.   18.
  13. ]
[  2.   1.   3.  nan   4.   6.   0.   5.   8.  24.  10.   7.  11.   9.  14.
  13.]


In [36]:
ques_cols_2 = ['rawcensustractandblock','censustractandblock']
for column in ques_cols_2:
    df = combined_df[column].unique()
    print(df)

[ 61110022.003007    61110015.031002    61110007.011007   ...,
  60375427.00302099  60379100.021083    60590218.152     ]
[  6.11100220e+13   6.11100150e+13   6.11100070e+13 ...,   6.03754270e+13
   6.03791000e+13   6.03790100e+13]


Here we can see that calculatedbathnbr contains float values, so we must treat it more like an object. But what I will do for now is to create a second categorical column for it without getting rid of the float column. Meanwhile we note that it suffices to keep the columns in ques_cols_2 as floats.

In [37]:
dfconverted = reduced_df.copy()
for col in int_cols:
    dfconverted[col] = dfconverted[col].fillna(-1.0).astype(int)
    
dfconverted['bathroom_number_category'] = str_to_intcat(dfconverted, 'calculatedbathnbr')
    
for col in cat_cols:
    dfconverted[col] = dfconverted[col].fillna(-1.0).astype(int)
    
for col in obj_cols:
    dfconverted[col] = str_to_intcat(dfconverted,col)
    
for col in date_cols:
    dfconverted[col] = pd.to_datetime(dfconverted[col], format='%Y.0', errors='coerce')
    
dfconverted['transactiondate'] = pd.to_datetime(dfconverted['transactiondate'], format = '%Y-%m-%d', errors = 'coerce')
    
dfconverted.head()

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,bathroom_number_category
0,17073783,-1,2,3.0,-1,2.5,1264.0,1264.0,6111,2,...,1986-01-01,115087.0,191811.0,2015-01-01,76724.0,2015.06,6.111002e+13,0.0953,2016-01-27,0
1,17088994,-1,1,2.0,-1,1.0,777.0,777.0,6111,1,...,1990-01-01,143809.0,239679.0,2015-01-01,95870.0,2581.30,6.111002e+13,0.0198,2016-03-30,1
2,17100444,-1,2,3.0,-1,2.0,1101.0,1101.0,6111,2,...,1956-01-01,33619.0,47853.0,2015-01-01,14234.0,591.64,6.111001e+13,0.0060,2016-05-27,2
3,17102429,-1,1,2.0,-1,1.5,1554.0,1554.0,6111,1,...,1965-01-01,45609.0,62914.0,2015-01-01,17305.0,682.78,6.111001e+13,-0.0566,2016-06-07,3
4,17109604,-1,2,4.0,-1,2.5,2415.0,2415.0,6111,2,...,1984-01-01,277000.0,554000.0,2015-01-01,277000.0,5886.92,6.111001e+13,0.0573,2016-08-08,0


We must also use multi-indexing, since some parcelid's are used in more than one unique transaction.

In [38]:
dfconverted = dfconverted.set_index(['parcelid',dfconverted.index])
dfconverted.head()

,,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,...,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,bathroom_number_category
parcelid,,,,,,,,,,,,,,,,,,,,,,
17073783,0,-1,2,3.0,-1,2.5,1264.0,1264.0,6111,2,2,...,1986-01-01,115087.0,191811.0,2015-01-01,76724.0,2015.06,6.111002e+13,0.0953,2016-01-27,0
17088994,1,-1,1,2.0,-1,1.0,777.0,777.0,6111,1,1,...,1990-01-01,143809.0,239679.0,2015-01-01,95870.0,2581.30,6.111002e+13,0.0198,2016-03-30,1
17100444,2,-1,2,3.0,-1,2.0,1101.0,1101.0,6111,2,2,...,1956-01-01,33619.0,47853.0,2015-01-01,14234.0,591.64,6.111001e+13,0.0060,2016-05-27,2
17102429,3,-1,1,2.0,-1,1.5,1554.0,1554.0,6111,1,2,...,1965-01-01,45609.0,62914.0,2015-01-01,17305.0,682.78,6.111001e+13,-0.0566,2016-06-07,3
17109604,4,-1,2,4.0,-1,2.5,2415.0,2415.0,6111,2,2,...,1984-01-01,277000.0,554000.0,2015-01-01,277000.0,5886.92,6.111001e+13,0.0573,2016-08-08,0


Now we're going to look at some of the columns which were dropped earlier which perhaps should not have been.

In particular, decktypeid, hashottuborspa, poolcnt, poolsizesum, pooltypeid10, pooltypeid2, pooltypeid7, yardbuildingsqft17, yardbuildingsqft26, fireplaceflag, taxdelinquencyflag, taxdelinquencyyear seem to represent values of zero or false, etc as null values. So we will need to rework these columns.

In [40]:
forgotten_cols = ['parcelid','decktypeid', 'hashottuborspa', 'poolcnt', 'poolsizesum', 
                  'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'yardbuildingsqft17', 
                  'yardbuildingsqft26', 'fireplacecnt', 'taxdelinquencyyear']
forgotten_df = combined_df[forgotten_cols].copy()
forgotten_df = forgotten_df.fillna(value = 0)


#we will deal first with the hottub and pool fields as they can be cleaned up considerably. 
#We can perhaps drop hashottub as well as combine the pooltypes and count into a single field.
spa_compare = forgotten_df['hashottuborspa'] + forgotten_df['pooltypeid10']
print(spa_compare.unique())
# This returns either nan or 2, which means they concur, so we can drop hashottuborspa

#Here we combine each pool type id into a single, larger pooltype
forgotten_df['pooltypeid'] = forgotten_df['pooltypeid10'] + 2*forgotten_df['pooltypeid2'] + 5*forgotten_df['pooltypeid7']
#forgotten_df['poolsizesum'] = forgotten_df['poolsizesum'].fillna(value = 0)

forgotten_df['pooltypeid'] = forgotten_df['pooltypeid'].astype(int)

forgotten_df = forgotten_df.drop(['hashottuborspa', 'poolcnt','pooltypeid10','pooltypeid2','pooltypeid7'], axis=1)

#Now we clean up a decktype
forgotten_df['decktypeid']= forgotten_df['decktypeid'] / 60
forgotten_df['decktypeid']= forgotten_df['decktypeid'].astype(int)

forgotten_df['fireplacecnt'] = forgotten_df['fireplacecnt'].fillna(value = 0.0).astype(int)


#Finally, we convert taxdelinquencyyear to a proper datetime
def taxyear_todate(col):
    result = []
    for entry in forgotten_df[col]:
        if entry == 0:
            result.append(int(0))
        elif entry == 99:
            result.append(int(1999))
        else:
            result.append(int(2000+entry))
    return pd.Series((v for v in result))

forgotten_df['taxdelinquencyyear'] = taxyear_todate('taxdelinquencyyear')
forgotten_df['taxdelinquencyyear'] = pd.to_datetime(forgotten_df['taxdelinquencyyear'], 
                                                    format = '%Y', errors = 'coerce')

forgotten_df = forgotten_df.set_index(['parcelid',forgotten_df.index])
forgotten_df.describe()

[ 0.  2.  1.]


,decktypeid,poolsizesum,yardbuildingsqft17,yardbuildingsqft26,fireplacecnt,pooltypeid
count,167805.000000,167805.000000,167805.000000,167805.000000,167805.000000,167805.000000
mean,0.007580,5.683949,9.246471,0.266726,0.126850,0.983654
std,0.086734,56.416211,65.685803,12.511396,0.400143,1.957168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1750.000000,3191.000000,1366.000000,5.000000,5.000000


Now it's time to rejoin these columns to the original data

In [41]:
cleaned_df = dfconverted.join(forgotten_df, how = 'outer')
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 167805 entries, (17073783, 0) to (14737537, 77578)
Data columns (total 42 columns):
airconditioningtypeid           167805 non-null int64
bathroomcnt                     167805 non-null int64
bedroomcnt                      167805 non-null float64
buildingqualitytypeid           167805 non-null int64
calculatedbathnbr               166056 non-null float64
calculatedfinishedsquarefeet    166992 non-null float64
finishedsquarefeet12            159519 non-null float64
fips                            167805 non-null int64
fullbathcnt                     167805 non-null int64
garagecarcnt                    167805 non-null int64
garagetotalsqft                 55457 non-null float64
heatingorsystemtypeid           167805 non-null int64
latitude                        167805 non-null float64
longitude                       167805 non-null float64
lotsizesquarefeet               149427 non-null float64
propertycountylandusecode       167805 no

In [42]:
# Now, lets just rearrange the columns a bit

cols = cleaned_df.columns.tolist()
cols.pop(cols.index('transactiondate'))
cols.pop(cols.index('logerror'))
cols = cols+['transactiondate','logerror']
cleaned_df = cleaned_df[cols]
cleaned_df.head()

,,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,...,bathroom_number_category,decktypeid,poolsizesum,yardbuildingsqft17,yardbuildingsqft26,fireplacecnt,taxdelinquencyyear,pooltypeid,transactiondate,logerror
parcelid,,,,,,,,,,,,,,,,,,,,,,
17073783,0,-1,2,3.0,-1,2.5,1264.0,1264.0,6111,2,2,...,0,0,0.0,128.0,0.0,0,NaT,0,2016-01-27,0.0953
17088994,1,-1,1,2.0,-1,1.0,777.0,777.0,6111,1,1,...,1,0,0.0,198.0,0.0,0,NaT,0,2016-03-30,0.0198
17100444,2,-1,2,3.0,-1,2.0,1101.0,1101.0,6111,2,2,...,2,0,0.0,0.0,0.0,0,NaT,0,2016-05-27,0.0060
17102429,3,-1,1,2.0,-1,1.5,1554.0,1554.0,6111,1,2,...,3,0,0.0,0.0,0.0,1,NaT,0,2016-06-07,-0.0566
17109604,4,-1,2,4.0,-1,2.5,2415.0,2415.0,6111,2,2,...,0,0,0.0,0.0,0.0,1,NaT,0,2016-08-08,0.0573


In [43]:
cleaned_df.to_csv('cleaned_1.csv')